In [9]:

from functools import wraps
import os
import time
import traceback
import signal

def _wrap_with_strace(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        import os
        import time
        import traceback
        import subprocess
        import signal
        
        parent_pid = os.getpid()
        child_pid = os.fork()
        
        if child_pid == 0:    
            p = subprocess.run(
                [
                    "sudo", "-S",  # -S allows reading the password from stdin
                    "strace",
                    "-qqq",
                    "-r",
                    "-z",
                    "-f",
                    "-o", "/home/admin/Talha/strace.txt",
                    "-p", str(parent_pid),
                ],
                input="ADD_PASSWORD".encode(),
                check=True,  # Raise an exception if the command fails
            )
        elif child_pid > 0:
            try:
                time.sleep(1)
                result = func(*args, **kwargs)
                os.kill(child_pid, signal.SIGTERM)
                s = os.waitpid(child_pid, 0)
        
                return result
            except Exception as e:
                print(traceback.format_exc())
                os._exit(1)
        else:
            # Parent process could not fork
            print(traceback.format_exc())
            os._exit(1)
        return result
    return wrapper

    
import ast

class WrapVineCalls(ast.NodeTransformer):
    def visit_Call(self, node):
        # Check if this is a call to vine.PythonTask
        if isinstance(node.func, ast.Attribute) and node.func.attr == 'PythonTask':
            if isinstance(node.func.value, ast.Name) and node.func.value.id == 'vine':
                # Ensure there’s at least one argument                
                if len(node.args) >= 1:
                    original_func = node.args[0]  # e.g., multiply_pair
                    # Create a new Call node: _wrap_with_prints(original_func)
                    wrapped_func = ast.Call(
                        func=ast.Name(id='_wrap_with_strace', ctx=ast.Load()),
                        args=[original_func],
                        keywords=[]
                    )
                    # Replace the first argument with the wrapped version
                    node.args[0] = wrapped_func
        # Continue traversing the AST
        return self.generic_visit(node)

ip = get_ipython()
ip.user_ns['_wrap_with_strace'] = _wrap_with_strace
ip.ast_transformers.append(WrapVineCalls())

In [10]:
def multiply_pair(A, B):

    import numpy as np  # Only on the worker
    A_np = np.array(A, dtype=float)
    B_np = np.array(B, dtype=float)
    C_np = A_np @ B_np
    return C_np.tolist()  # convert back to nested list
    # return 10




In [11]:
import os

manager_name = api_key = os.environ.get("VINE_MANAGER_NAME")
print(f"Manager name: {manager_name}")

Manager name: None


In [12]:
import random
import ndcctools.taskvine as vine

N = 3      # 50x50 matrices
num_pairs = 2  # We'll do 10 pairs for demonstration

A_list = []
B_list = []

for _ in range(num_pairs):
    A_mat = [[random.random() for _ in range(N)] for _ in range(N)]
    B_mat = [[random.random() for _ in range(N)] for _ in range(N)]
    A_list.append(A_mat)
    B_list.append(B_mat)

m = vine.Manager([9123,9150], name="matrix")

print(f"[manager] Listening on port {m.port}")

tasks_map = {}
results = [None]*num_pairs


workers = vine.Factory(manager=m)
workers.max_workers = 1
workers.min_workers = 1
workers.cores = 1
workers.memory = 1000
workers.disk = 1000

for i in range(num_pairs):
    task = vine.PythonTask(multiply_pair, A_list[i], B_list[i])
    t_id = m.submit(task)
    tasks_map[t_id] = i
    print(f"[manager] Submitted multiplication for pair index {i}.")

print("[manager] Waiting for tasks to complete...")
with workers:
   
    while not m.empty():
        done_task = m.wait(5)
        if done_task:
            idx = tasks_map[done_task.id]
            if done_task.successful():
                C = done_task.output
                results[idx] = C
                print(f"[manager] Pair {idx} -> result row0 length = {C}")
            else:
                print(f"[manager] Task for pair {idx} failed: {done_task.result}")
    
    print("\n[manager] All tasks done.")

[manager] Listening on port 9126
[manager] Submitted multiplication for pair index 0.
[manager] Submitted multiplication for pair index 1.
[manager] Waiting for tasks to complete...
[manager] Pair 0 -> result row0 length = [[0.5885234552684022, 0.5836761752202477, 0.8248885602697322], [0.9537716414713713, 1.1559984385961575, 1.2548068582380234], [0.7379267386976747, 0.8049917673851577, 1.0915167310681837]]
[manager] Pair 1 -> result row0 length = [[1.8628131629264715, 1.4055611906309764, 1.8266625757204604], [0.8707992001067214, 0.48850317983564573, 0.6483068676151337], [1.1177419538077409, 0.9216723542550617, 1.1753120502392378]]

[manager] All tasks done.
